In [111]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time


client = Client("https://api.devnet.solana.com")


while not client.is_connected():
    time.sleep(60)

In [112]:
program_id = PublicKey("7fmmvCJ1uH4sfLCdZrDAjJnfHejwtFbYL7aG8nnNR6Yz")
dsol_mint = PublicKey("HSYdApQF2FKVNnX6FQ1ySYstGLe5Gp8NseHVFavWye66")

seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_seed(seed)
(sales_pda, sales_nonce) = PublicKey.find_program_address([b'table_sales_pda'], program_id)
# client.request_airdrop(payer_keypair.public_key, 2*10**9)
payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
sales_pda_meta = AccountMeta(sales_pda, False, True)

vault_pda, _ = PublicKey.find_program_address([b"Dsol_vault_tables"], program_id)
vault_account_meta = AccountMeta(vault_pda, False, True)
vault_pda_dsol_token_account_meta = AccountMeta(instructions.get_associated_token_address(vault_pda, dsol_mint), False, True)

(pending_pda, pending_nonce) = PublicKey.find_program_address([b"pending_tables"], program_id)
pending_tables_meta = AccountMeta(pending_pda, False, True)

main_vault_key = PublicKey("G473EkeR5gowVn8CRwTSDop3zPwaNixwp62qi7nyVf4z")
main_vault_assoc = instructions.get_associated_token_address(main_vault_key, dsol_mint)

main_vault_assoc_meta = AccountMeta(main_vault_assoc, False, True)
main_vault_meta = AccountMeta(main_vault_key, False, True)

sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
dsol_mint_meta = AccountMeta(dsol_mint, False, False)
associated_program_meta = AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False)

accounts = [
    payer_account_meta,
    sales_pda_meta,
    pending_tables_meta,
    vault_pda_dsol_token_account_meta,
    vault_account_meta,
    main_vault_assoc_meta,
    main_vault_meta,
    dsol_mint_meta,
    sys_program_meta,
    spl_program_meta,
    rent_account_meta,


    associated_program_meta,
    associated_program_meta,
    sys_program_meta,
    sys_program_meta,
    sys_program_meta,
    sys_program_meta
]


In [113]:
[act.pubkey for act in accounts]

[2ASw3tjK5bSxQxFEMsM6J3DnBozNh7drVErSwc7AtzJv,
 3RbZo6tvDdZJ78DYqMR4nxftSkjt91eRZgYSNLjfAuhw,
 4rr4wFqxD6dw1o2Ud9dEhSDQHHBYVYdRaoYFq1d35kWz,
 9ztksFgqh92Z1C9EcUGRDcV7D8aQfxEDdLqV19G3tRUf,
 ABd6dVBZHQbB4VrW1tahFETcfVmXtqGrKgzTgdWjhrFL,
 H5bXtRguFwmif1VLQAVcYZwA5UwBQ8qUMders9TXLiQ7,
 G473EkeR5gowVn8CRwTSDop3zPwaNixwp62qi7nyVf4z,
 HSYdApQF2FKVNnX6FQ1ySYstGLe5Gp8NseHVFavWye66,
 11111111111111111111111111111111,
 TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,
 SysvarRent111111111111111111111111111111111,
 ATokenGPvbdGVxr1b2hvZbsiqW5xWH25efTNsLJA8knL,
 ATokenGPvbdGVxr1b2hvZbsiqW5xWH25efTNsLJA8knL,
 11111111111111111111111111111111,
 11111111111111111111111111111111,
 11111111111111111111111111111111,
 11111111111111111111111111111111]

In [114]:
payer_keypair.public_key

2ASw3tjK5bSxQxFEMsM6J3DnBozNh7drVErSwc7AtzJv

In [115]:
InstructionEnum = Enum(
    "InitTable" / CStruct("governor_reward" / U32, "payer_bump" / U32),
    "SignTableMint",
    "CreateTableSalesAccount",
    "BurnNFTs" / CStruct("rarity" / U8),
    "MintTable",
    enum_name = "InstructionEnum"
)

In [116]:
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    InstructionEnum.build(InstructionEnum.enum.CreateTableSalesAccount())
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/NRTv5o4i5isn39gY4DbgnJdFGmgDWz4rF9q23qmjtA8oYhPJ4ES1JrwPKxZ2acNrWu7qcUofCutuYkoAj7JroTx?cluster=devnet'